In [ ]:
import cv2
import moviepy.editor as mpy
from IPython.display import Video
import cv2
import ipywidgets as widgets
from IPython.display import Video, display

In [ ]:
import cv2
import moviepy.editor as mpy

def manual_selection_and_extraction(video_path):
    """
    Combines manual selection of movements using OpenCV and extraction using MoviePy.

    Args:
        video_path: Path to the input video file.
    """

    cap = cv2.VideoCapture(video_path)

    timestamps = []
    start_time = None

    while True:
        ret, frame = cap.read()

        if not ret:
            break

        cv2.imshow('Video', frame)

        key = cv2.waitKey(1) & 0xFF

        if key == ord('s'):  # Start selection
            if start_time is None:
                start_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert to seconds
                print("Start time:", start_time)
            else:
                print("Start time already set. Press 'e' to end selection.")
        elif key == ord('e'):  # End selection
            if start_time is not None:
                end_time = cap.get(cv2.CAP_PROP_POS_MSEC) / 1000  # Convert to seconds
                print("End time:", end_time)
                timestamps.append((start_time, end_time))
                start_time = None
        elif key == ord('q'):  # Quit
            break

    cap.release()
    cv2.destroyAllWindows()

    # Extract and save clips using MoviePy
    clip = mpy.VideoFileClip(video_path)
    for start, end in timestamps:
        subclip = clip.subclip(start, end)
        output_path = f"clip_{start}_{end}.mp4"
        subclip.write_videofile(output_path)

# Example usage:
video_path = "your_video.mp4"
manual_selection_and_extraction(video_path)

In [ ]:
def play_video_and_save_timestamps(video_path):
    """
    Plays a video and allows user to manually select start and end times for movements using interactive widgets.

    Args:
        video_path: Path to the input video file.

    Returns:
        A list of tuples, where each tuple contains (start_time, end_time) of a selected movement.
    """

    cap = cv2.VideoCapture(video_path)

    timestamps = []
    start_time = None

    # Create interactive widgets
    video_widget = Video(data=video_path, autoplay=False)
    slider = widgets.IntSlider(min=0, max=int(cap.get(cv2.CAP_PROP_FRAME_COUNT)) - 1, step=1, value=0)
    start_button = widgets.Button(description='Start Selection')
    end_button = widgets.Button(description='End Selection')

    def on_slider_change(change):
        cap.set(cv2.CAP_PROP_POS_FRAMES, change['new'])
        ret, frame = cap.read()
        video_widget.value = frame

    def on_start_button_click(b):
        nonlocal start_time
        start_time = slider.value

    def on_end_button_click(b):
        nonlocal start_time
        if start_time is not None:
            end_time = slider.value
            timestamps.append((start_time, end_time))
            start_time = None

    slider.observe(on_slider_change, names='value')
    start_button.on_click(on_start_button_click)
    end_button.on_click(on_end_button_click)

    display(video_widget, slider, start_button, end_button)

    return timestamps

# Example usage:
video_path = "your_video.mp4"
timestamps = play_video_and_save_timestamps(video_path)

print("Selected timestamps:", timestamps)